In [1]:
using Revise
using BenchmarkTools

In [ ]:
using DataIO
using JSON

using GCTGMT
using Support

using FeatureSetEnrichment

In [ ]:
benchmarks_directory_path = "/Users/kwat/Downloads/score_and_set/"

benchmark_ = []

method_ = []

benchmark_x_method = []

In [ ]:
for name in readdir(benchmarks_directory_path)[1:2]
    
    if name[1] == '.'

        continue
        
    end

    benchmark_directory_path = joinpath(benchmarks_directory_path, name)

    element_, score_ = eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = convert(Dict{String, Vector{String}}, JSON.parse(open(joinpath(benchmark_directory_path, "gene_set.json"))))

    gmt_path_ = map(gmt_path -> replace(gmt_path, ".." => "/Users/kwat/Downloads/"), json_dict["gene_sets_tested"][1:2])

    set_to_element_ = read_gmt(gmt_path_)

    set_ = collect(keys(set_to_element_))

    n_set = length(set_)

    set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

    # Todo: get method only once at the top
    method_ = collect(keys(collect(values(set_to_method_to_result))[1]))

    set_x_method = Matrix(undef, n_set, length(method_))

    for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

        set_x_method_row = []

        for result in values(method_to_result)

            append!(set_x_method_row, result[2])

        end

        set_x_method[set_i, :] = set_x_method_row

    end

    for set in json_dict["gene_sets_positive"]

        push!(benchmark_, "$name.$set")

        benchmark_x_method_row = []

        for (method_i, set_score_) in enumerate(eachcol(set_x_method))

            set_score_, set_ = sort_like((Float64.(set_score_), set_))

            push!(benchmark_x_method_row, findfirst(set_ .== set) / n_set)

        end

        push!(benchmark_x_method, Float64.(benchmark_x_method_row))

    end

end

In [ ]:
benchmark_x_method